# 메모리 네트워크 이해.
End-to-End Memory Network를 구현해 보고, 이를 이용해서 bAbI 태스크를 직접 수행할 것.<br>
>목표
> * 두 개 이상의 입력을 받는 모델을 설계해본다.
> * 메모리 구조를 사용하는 메모리 네트워크에 대해서 이해한다.


---

## 1. 데이터셋 로드
라이브러리를 호출한 뒤 bAbl 데이터셋을 로드할 것.

In [ ]:
from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import tarfile
from nltk import FreqDist
from functools import reduce
import os
import re

tf.keras의 get_file()을 통해 다운로드.

In [ ]:
home_dir = os.getenv('HOME')+'/Aiffel_Project/Dataset/Babi_memory_net'
file_to_save = home_dir + '/babi-tasks-v1-2.tar.gz'
path = get_file(file_to_save, origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
print(path)

압축 파일이기 때문에 압축을 해제

In [ ]:
with tarfile.open(path) as tar:
    tar.extractall(home_dir)  # ~/aiffel/babi_memory_net 아래에 압축해제
    tar.close()

훈련 데이터의 경로와 테스트 데이터의 경로를 지정

In [ ]:
DATA_DIR = home_dir + '/tasks_1-20_v1-2/en-10k'
TRAIN_FILE = os.path.join(DATA_DIR, "qa1_single-supporting-fact_train.txt")
TEST_FILE = os.path.join(DATA_DIR, "qa1_single-supporting-fact_test.txt")

훈련 데이터에서 20개의 문장을 출력.

In [ ]:
i = 0
lines = open(TRAIN_FILE , "rb")
for line in lines:
    line = line.decode("utf-8").strip()
    # lno, text = line.split(" ", 1) # ID와 TEXT 분리
    i = i + 1
    print(line)
    if i == 20:
        break

## 2. 데이터 전처리
첫 번째 전처리는 데이터를 읽는 과정에서 스토리, 질문, 답변을 각각 분리해서 저장.<br>
supporting fact(실제 정답이 몇번째 문장에 있었는지를 알려주는 인덱스 힌트 정보)는 저장하지 않을 것.

In [ ]:
def read_data(dir):
    stories, questions, answers = [], [], [] # 각각 스토리, 질문, 답변을 저장할 예정
    story_temp = [] # 현재 시점의 스토리 임시 저장
    lines = open(dir, "rb")

    for line in lines:
        line = line.decode("utf-8") # b' 제거
        line = line.strip() # '\n' 제거
        idx, text = line.split(" ", 1) # 맨 앞에 있는 id number 분리
        # 여기까지는 모든 줄에 적용되는 전처리

        if int(idx) == 1:
            story_temp = []
        
        if "\t" in text: # 현재 읽는 줄이 질문 (tab) 답변 (tab)인 경우
            question, answer, _ = text.split("\t") # 질문과 답변을 각각 저장
            stories.append([x for x in story_temp if x]) # 지금까지의 누적 스토리를 스토리에 저장
            questions.append(question)
            answers.append(answer)

        else: # 현재 읽는 줄이 스토리인 경우
            story_temp.append(text) # 임시 저장

    lines.close()
    return stories, questions, answers

read_data()를 통해 스토리, 질문, 답변의 쌍을 리턴하여 훈련 데이터는 train_data()에, 테스트 데이터는 test_data()에 저장함.<br>
두 데이터를 가지고 추가적인 전처리를 진행할 예정.

In [ ]:
train_data = read_data(TRAIN_FILE)
test_data = read_data(TEST_FILE)

확인하기 쉽도록 스토리, 질문, 답변을 각각 저장하고 직접 출력해 볼 것.

In [ ]:
train_stories, train_questions, train_answers = read_data(TRAIN_FILE)
test_stories, test_questions, test_answers = read_data(TEST_FILE)

각각의 샘플 개수 출력

In [ ]:
print("train 스토리 개수:", len(train_stories))
print("train 질문 개수:", len(train_questions))
print("train 답변 개수:", len(train_answers))
print("test 스토리 개수:", len(test_stories))
print("test 질문 개수:", len(test_questions))
print("test 답변 개수:", len(test_answers))

훈련 데이터는 10,000개 테스트 데이터는 1,000개임을 알 수 있음.<br><br>

임의로 3,879번째 스토리를 출력.



In [ ]:
train_stories[3878]

상위 5개의 질문을 출력.

In [ ]:
train_questions[:5]

상위 5개의 답변을 출력

In [ ]:
train_answers[:5]

### 토큰화를 위한 함수를 생성

In [ ]:
def tokenize(sent):
    return [ x.strip() for x in re.sub(r"\s+|\b", '\f', sent).split('\f') if x.strip() ] # python 3.7의 경우 
    # return [ x.strip() for x in re.split('(\W+)?', sent) if x.strip()] # python 3.6의 경우

### 단어장을 생성하고, 단어에서 정수로, 정수에서 단어로 맵핑하는 딕셔너리(dictionary)를 생성.
스토리와 질문의 가장 긴 길이를 구해서 padding 적용 과정에 이용할 것.<br><br>


단, 전처리를 하면서 같은 스토리 내의 여러 문장들 하나의 문장으로 통합할 것.<br>
가령, 앞서 3,879번째 스토리는 8개의 문장으로 구성되어 있으나 전처리 과정에서 8개의 문장을 모두 이어 붙여서 1개의 문장으로 통합.

In [ ]:
def preprocess_data(train_data, test_data):
    counter = FreqDist()
    
    # 두 문장의 story를 하나의 문장으로 통합하는 함수
    flatten = lambda data: reduce(lambda x, y: x + y, data)

    # 각 샘플의 길이를 저장하는 리스트
    story_len = []
    question_len = []
    
    for stories, questions, answers in [train_data, test_data]:
        for story in stories:
            stories = tokenize(flatten(story)) # 스토리의 문장들을 펼친 후 토큰화
            story_len.append(len(stories)) # 각 story의 길이 저장
            for word in stories: # 단어 집합에 단어 추가
                counter[word] += 1
        for question in questions:
            question = tokenize(question)
            question_len.append(len(question))
            for word in question:
                counter[word] += 1
        for answer in answers:
            answer = tokenize(answer)
            for word in answer:
                counter[word] += 1

    # 단어장 생성
    word2idx = {word : (idx + 1) for idx, (word, _) in enumerate(counter.most_common())}
    idx2word = {idx : word for word, idx in word2idx.items()}

    # 가장 긴 샘플의 길이
    story_max_len = np.max(story_len)
    question_max_len = np.max(question_len)

    return word2idx, idx2word, story_max_len, question_max_len

전처리 함수를 사용하여 단어장과 가장 긴 샘플의 길이를 리턴 

In [ ]:
word2idx, idx2word, story_max_len, question_max_len = preprocess_data(train_data, test_data)

단어장을 출력

In [ ]:
print(word2idx)

실제 변수로 사용할 단어장의 크기는 패딩을 고려하여 +1 을 해주어야 함.

In [ ]:
vocab_size = len(word2idx) + 1

전처리 함수를 통해 구한 스토리와 질문의 최대 길이 출력

In [ ]:
print('스토리의 최대 길이 :',story_max_len)
print('질문의 최대 길이 :',question_max_len)

* 스토리의 최대 길이는 68가 나옴.  이는 스토리 내에 있는 여러 개 문장들을 하나의 문장으로 간주하였을 때, 최대 단어의 개수.<br>
* 질문의 최대 길이는 4. bAbI 데이터셋에는 대체로 'Where is Mary?'와 같은 매우 짧은 길이의 질문들만 존재한다는 의미.

### 남은 전처리를 진행하는 함수 생성

* 텍스트 데이터를 단어와 맵핑되는 정수로 인코딩. 이 과정은 앞서 만들어놓은 word2idx를 활용.
* 스토리와 질문 데이터에 대해서 각각의 최대 길이로 패딩(padding). 이 과정은 앞서 계산해놓은 story_max_len과 question_max_len을 사용.
* 레이블에 해당되는 정답 데이터를 원-핫 인코딩.

In [ ]:
def vectorize(data, word2idx, story_maxlen, question_maxlen):
    Xs, Xq, Y = [], [], []
    flatten = lambda data: reduce(lambda x, y: x + y, data)

    stories, questions, answers = data
    for story, question, answer in zip(stories, questions, answers):
        xs = [word2idx[w] for w in tokenize(flatten(story))]
        xq = [word2idx[w] for w in tokenize(question)]
        Xs.append(xs)
        Xq.append(xq)
        Y.append(word2idx[answer])

    # 스토리와 질문은 각각의 최대 길이로 패딩
    # 정답은 원-핫 인코딩
    return pad_sequences(Xs, maxlen=story_maxlen),\
           pad_sequences(Xq, maxlen=question_maxlen),\
           to_categorical(Y, num_classes=len(word2idx) + 1)

훈련 데이터와 테스트 데이터에 적용

In [ ]:
Xstrain, Xqtrain, Ytrain = vectorize(train_data, word2idx, story_max_len, question_max_len)
Xstest, Xqtest, Ytest = vectorize(test_data, word2idx, story_max_len, question_max_len)

반환된 결과의 크기(shape)를 확인

In [ ]:
print(Xstrain.shape, Xqtrain.shape, Ytrain.shape, Xstest.shape, Xqtest.shape, Ytest.shape)

## 3. 메모리 네트워크 구현하기

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Permute, dot, add, concatenate
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Activation
import matplotlib.pyplot as plt

하이퍼파라미터를 정의

In [ ]:
# 에포크 횟수
train_epochs = 120
# 배치 크기
batch_size = 32
# 임베딩 크기
embed_size = 50
# LSTM의 크기
lstm_size = 64
# 과적합 방지 기법인 드롭아웃 적용 비율
dropout_rate = 0.30

입력을 담아두는 변수들을 정의

In [ ]:
input_sequence = Input((story_max_len,))
question = Input((question_max_len,))
 
print('Stories :', input_sequence)
print('Question:', question)

#### 먼저 텍스트 입력을 임베딩으로 변환하는 인코더를 구현
![embedding encoder](embedding_encoder.png)

In [ ]:
# 스토리를 위한 첫 번째 임베딩. 그림에서의 Embedding A
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=embed_size))
input_encoder_m.add(Dropout(dropout_rate))
# 결과 : (samples, story_max_len, embed_size) / 샘플의 수, 문장의 최대 길이, 임베딩 벡터의 차원
 
# 스토리를 위한 두 번째 임베딩. 그림에서의 Embedding C
# 임베딩 벡터의 차원을 question_max_len(질문의 최대 길이)로 한다.
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=question_max_len))
input_encoder_c.add(Dropout(dropout_rate))
# 결과 : (samples, story_max_len, question_max_len) / 샘플의 수, 문장의 최대 길이, 질문의 최대 길이(임베딩 벡터의 차원)

In [ ]:
# 질문을 위한 임베딩. 그림에서의 Embedding B
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=embed_size,
                               input_length=question_max_len))
question_encoder.add(Dropout(dropout_rate))
# 결과 : (samples, question_max_len, embed_size) / 샘플의 수, 질문의 최대 길이, 임베딩 벡터의 차원

#### 구현된 인코더를 이용해 텍스트를 임베딩

In [ ]:
# 실질적인 임베딩 과정
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

print('Input encoded m', input_encoded_m, '\n')
print('Input encoded c', input_encoded_c, '\n')
print('Question encoded', question_encoded, '\n')

###  스토리 문장과 질문 문장의 매칭 유사도 계산 구현
![soft attention 방식의 유사도 구하기](soft_attention.png)


<center>(soft attention 방식의 유사도 구하기)</center>


$$ p = Softmax(dot(m,u)) $$

In [ ]:
# 스토리 단어들과 질문 단어들 간의 유사도를 구하는 과정
# 유사도는 내적을 사용한다.
match = dot([input_encoded_m, question_encoded], axes=-1, normalize=False)
match = Activation('softmax')(match)
print('Match shape', match)
# 결과 : (samples, story_max_len, question_max_len) / 샘플의 수, 문장의 최대 길이, 질문의 최대 길이

예측에 사용되는 출력 행렬은 매칭 유사도 match와 스토리 표현 input_encoded_c을 더해서 도출.

In [ ]:
# 매칭 유사도 행렬과 질문에 대한 임베딩을 더한다.
response = add([match, input_encoded_c])  # (samples, story_maxlen, question_max_len)
response = Permute((2, 1))(response)  # (samples, question_max_len, story_maxlen)
print('Response shape', response)

## 4. 모델 정의 및 훈련 수행

In [ ]:
# concatenate the response vector with the question vector sequence
answer = concatenate([response, question_encoded])
print('Answer shape', answer)
 
answer = LSTM(lstm_size)(answer)  # Generate tensors of shape 32
answer = Dropout(dropout_rate)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

In [ ]:
import os

# 모델 컴파일
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['acc'])
 
# 테스트 데이터를 검증 데이터로 사용하면서 모델 훈련 시작
history = model.fit([Xstrain, Xqtrain],
         Ytrain, batch_size, train_epochs,
         validation_data=([Xstest, Xqtest], Ytest))
 
# 훈련 후에는 모델 저장
model_path = os.getenv('HOME')+'/aiffel/babi_memory_net/model.h5'
model.save(model_path)

테스트 정확도를 출력

In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate([Xstest, Xqtest], Ytest)[1]))

#### 훈련 과정에서 기록해두었던 훈련 데이터, 검증 데이터의 정확도와 loss를 그래프로 출력

In [ ]:
# plot accuracy and loss plot
plt.subplot(211)
plt.title("Accuracy")
plt.plot(history.history["acc"], color="g", label="train")
plt.plot(history.history["val_acc"], color="b", label="validation")
plt.legend(loc="best")

plt.subplot(212)
plt.title("Loss")
plt.plot(history.history["loss"], color="g", label="train")
plt.plot(history.history["val_loss"], color="b", label="validation")
plt.legend(loc="best")

plt.tight_layout()
plt.show()

# labels
ytest = np.argmax(Ytest, axis=1)

# get predictions
Ytest_ = model.predict([Xstest, Xqtest])
ytest_ = np.argmax(Ytest_, axis=1)

테스트 데이터에 대해 실제로 문제를 잘 맞추는지 임의로 예측 결과 30개를 출력

In [ ]:
NUM_DISPLAY = 30

print("{:20}|{:7}|{}".format("질문", "실제값", "예측값"))
print(39 * "-")

for i in range(NUM_DISPLAY):
    question = " ".join([idx2word[x] for x in Xqtest[i].tolist()])
    label = idx2word[ytest[i]]
    prediction = idx2word[ytest_[i]]
    print("{:20}: {:8} {}".format(question, label, prediction))